In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.data_paths import *

In [5]:
df = pd.read_pickle(MERGED)

In [6]:
cosmic_proteins = pd.read_csv(COSMIC_PROTEINS, sep='\t')

In [10]:
df['name_y'].dropna()

1172       gag-pol
1173       gag-pol
1174       gag-pol
1175       gag-pol
1176       gag-pol
            ...   
2903162       CDK9
2903163       CDK9
2903177       gyrB
2903178       gyrB
2903179       gyrB
Name: name_y, Length: 94083, dtype: object

In [20]:
cosmic_proteins

,Gene,Gene synonym,Ensembl,Gene description,Uniprot,Chromosome,Position,Protein class,Biological process,Molecular function,...,Cancer prognostics - Pancreatic Adenocarcinoma (TCGA),Cancer prognostics - Pancreatic Adenocarcinoma (validation),Cancer prognostics - Prostate Adenocarcinoma (TCGA),Cancer prognostics - Rectum Adenocarcinoma (TCGA),Cancer prognostics - Rectum Adenocarcinoma (validation),Cancer prognostics - Skin Cuteneous Melanoma (TCGA),Cancer prognostics - Stomach Adenocarcinoma (TCGA),Cancer prognostics - Testicular Germ Cell Tumor (TCGA),Cancer prognostics - Thyroid Carcinoma (TCGA),Cancer prognostics - Uterine Corpus Endometrial Carcinoma (TCGA)
0,ABI1,"ABI-1, E3B1, SSH3BP1",ENSG00000136754,Abl interactor 1,Q8IZP0,10,26746593-26861087,"Cancer-related genes, Disease related genes, P...",Host-virus interaction,NaN,...,unprognostic (6.17e-3),unprognostic (9.66e-2),unprognostic (8.60e-2),unprognostic (1.16e-1),unprognostic (3.28e-2),unprognostic (1.01e-1),unprognostic (3.66e-1),unprognostic (1.72e-1),unprognostic (1.03e-1),unprognostic (1.72e-1)
1,ABL1,"ABL, c-ABL, JTK7, p150",ENSG00000097007,"ABL proto-oncogene 1, non-receptor tyrosine ki...",P00519,9,130713043-130887675,"Cancer-related genes, Disease related genes, E...","Apoptosis, Autophagy, Cell adhesion, DNA damag...","DNA-binding, Kinase, Transferase, Tyrosine-pro...",...,unprognostic (6.12e-2),unprognostic (3.31e-2),unprognostic (1.27e-1),unprognostic (2.01e-2),unprognostic (2.18e-1),unprognostic (2.61e-1),unprognostic (8.92e-2),unprognostic (3.15e-1),unprognostic (1.29e-1),unprognostic (1.96e-2)
2,ABL2,"ABLL, ARG",ENSG00000143322,"ABL proto-oncogene 2, non-receptor tyrosine ki...",P42684,1,179099330-179229684,"Cancer-related genes, Enzymes, Predicted intra...",Cell adhesion,"Kinase, Transferase, Tyrosine-protein kinase",...,unprognostic (2.27e-2),unprognostic (1.90e-3),unprognostic (1.86e-1),unprognostic (1.08e-1),unprognostic (7.21e-2),unprognostic (2.24e-1),unprognostic (9.34e-2),unprognostic (1.59e-1),unprognostic (2.71e-2),unprognostic (5.39e-2)
3,ACKR3,"CMKOR1, CXCR7, GPR159, RDC1",ENSG00000144476,Atypical chemokine receptor 3,P25106,2,236567787-236582354,"Cancer-related genes, Disease related genes, G...","Cell adhesion, Host-virus interaction","Developmental protein, G-protein coupled recep...",...,unprognostic (1.48e-3),unprognostic (8.51e-2),unprognostic (3.54e-2),unprognostic (1.21e-1),unprognostic (4.76e-3),unprognostic (1.29e-2),unprognostic (1.95e-4),unprognostic (1.41e-1),unprognostic (1.15e-3),unprognostic (4.99e-2)
4,ACSL3,"ACS3, FACL3, PRO2194",ENSG00000123983,Acyl-CoA synthetase long chain family member 3,O95573,2,222860942-222944639,"Cancer-related genes, Enzymes, Metabolic prote...","Fatty acid metabolism, Lipid metabolism",Ligase,...,unprognostic (7.65e-2),unprognostic (1.24e-1),unprognostic (2.88e-2),unprognostic (3.03e-1),unprognostic (2.64e-2),unprognostic (8.26e-2),unprognostic (9.29e-2),unprognostic (1.77e-1),unprognostic (1.08e-1),unprognostic (2.19e-1)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,ZMYM2,"FIM, MYM, RAMP, ZNF198",ENSG00000121741,Zinc finger MYM-type containing 2,Q9UBW7,13,19958677-20091829,"Cancer-related genes, Disease related genes, H...","Transcription, Transcription regulation",NaN,...,unprognostic (3.57e-2),unprognostic (4.85e-1),unprognostic (1.21e-1),unprognostic (4.77e-2),unprognostic (1.74e-1),unprognostic (7.15e-2),unprognostic (2.15e-1),unprognostic (1.08e-1),unprognostic (3.70e-3),unprognostic (1.30e-1)
568,ZNF331,"RITA, ZNF361, ZNF463",ENSG00000130844,Zinc finger protein 331,Q9NQX6,19,53519527-53580269,"Cancer-related genes, Predicted intracellular ...","Transcription, Transcription regulation",DNA-binding,...,unprognostic (1.35e-2),unprognostic (4.71e-2),unprognostic (4.12e-1),unprognostic (2.69e-1),unprognostic (5.47e-3),unprognostic (1.28e-2),unprognostic (1.03e-3),unprognostic (1.42e-1),unprognostic (2.32e-4),unprognostic (3.34e-2)
569,ZNF384,"CAGH1A, C

In [17]:
cosmic_proteins['Gene']

0        ABI1
1        ABL1
2        ABL2
3       ACKR3
4       ACSL3
        ...  
567     ZMYM2
568    ZNF331
569    ZNF384
570    ZNF521
571     ZRSR2
Name: Gene, Length: 572, dtype: object

In [43]:
arr = cosmic_proteins['Gene synonym'].apply(lambda x: np.array([y.strip() for y in str(x).split(',')]))
all_elems = np.concatenate(arr)

# Check isin for each element in the array (careful of double lists)
df['name_y'].isin(all_elems).sum()

np.int64(0)

In [44]:
df['name_y'].isin(cosmic_proteins['Gene']).sum()

np.int64(20199)

In [45]:
cosmic_proteins['Gene'].isin(df['name_y']).sum()

np.int64(45)

In [46]:
df[df['name_y'].isin(cosmic_proteins['Gene'])].head(2)

,ki,ph,temp,ic50,ec50,kd,kon,koff,doi,target_name,...,chembl_id_DrugBank,smiles_DrugBank,inchi_key_DrugBank,chebi_id_DrugBank,bindingdb_id_DrugBank,Matched_On,name_y,general-function,specific-function,organism
8960,<NA>,7.6,22.0,840.0,<NA>,<NA>,<NA>,<NA>,10.1021/jm0201722,Cyclin-dependent kinase 4/G1/S-specific cyclin-D1,...,NaN,NaN,NaN,NaN,NaN,NaN,CCND1,Transcription factor binding,Regulatory component of the cyclin D1-CDK4 (DC...,Humans
8961,<NA>,7.6,22.0,490.0,<NA>,<NA>,<NA>,<NA>,10.1021/jm0201722,Cyclin-dependent kinase 4/G1/S-specific cyclin-D1,...,NaN,NaN,NaN,NaN,NaN,NaN,CCND1,Transcription factor binding,Regulatory component of the cyclin D1-CDK4 (DC...,Humans


In [49]:
cancer_proteins = df[df['name_y'].isin(cosmic_proteins['Gene'])]

ligands_related_to_cancer_proteins = cancer_proteins['name_x'].unique()
len(ligands_related_to_cancer_proteins), ligands_related_to_cancer_proteins

(191,
 array([nan, 'Alvocidib', 'Olomoucine', 'G9ZF61LE7G',
        "N'-(Pyrrolidino[2,1-B]Isoindolin-4-On-8-Yl)-N-(Pyridin-2-Yl)Urea",
        '1-(5-OXO-2,3,5,9B-TETRAHYDRO-1H-PYRROLO[2,1-A]ISOINDOL-9-YL)-3-(5-PYRROLIDIN-2-YL-1H-PYRAZOL-3-YL)-UREA',
        'WS821Z52LQ', 'Pteroic acid', 'K-252a',
        '6,7,12,13-tetrahydro-5H-indolo[2,3-a]pyrrolo[3,4-c]carbazol-5-one',
        'SB-218078', 'Bisindolylmaleimide I', 'Staurosporine',
        '64H5U84UFB', 'NTZ47MG2A4',
        '4-[4-(4-Methyl-2-Methylamino-Thiazol-5-Yl)-Pyrimidin-2-Ylamino]-Phenol',
        '[4-(2-Amino-4-Methyl-Thiazol-5-Yl)-Pyrimidin-2-Yl]-(3-Nitro-Phenyl)-Amine',
        "O6-CYCLOHEXYLMETHOXY-2-(4'-SULPHAMOYLANILINO) PURINE",
        'T61871RKRI', 'Wortmannin', '7-Hydroxystaurosporine', 'Pictilisib',
        'TG100-115', 'LY-294002', 'Milciclib Maleate', 'Roginolisib',
        'Tirabrutinib', 'Navtemadlin', 'Y2SR66P7VM', 'Trilaciclib',
        'Lerociclib', 'AUY922', 'Vorasidenib', 'Enasidenib', 'Abemaciclib',
    